# Kickstarter Projects
#### More than 300,000 kickstarter projects
## About Dataset
### Context
I'm a crowdfunding enthusiast and i'm watching kickstarter since its early days. Right now I just collect data and the only app i've made is this twitter bot which tweet any project reaching some milestone: @bloomwatcher . I have a lot of other ideas, but sadly not enough time to develop them… But I hope you can!
### Content
You'll find most useful data for project analysis. Columns are self explanatory except:
- usd_pledged: conversion in US dollars of the pledged column (conversion done by kickstarter).
- usd pledge real: conversion in US dollars of the pledged column (conversion from Fixer.io API).
- usd goal real: conversion in US dollars of the goal column (conversion from Fixer.io API).
### Acknowledgements
Data are collected from Kickstarter Platform

usd conversion (usdpledgedreal and usdgoalreal columns) were generated from convert ks pledges to usd script done by tonyplaysguitar
### Inspiration
I hope to see great projects, and why not a model to predict if a project will be successful before it is released? :)

# First look at the data

In [172]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('data/ks-projects-201801.csv', index_col=0)

In [173]:
data.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
ID,,,,,,,,,,,,,,
1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [174]:
data.describe()

,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
count,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05
mean,4.908079e+04,9.682979e+03,105.617476,7.036729e+03,9.058924e+03,4.545440e+04
std,1.183391e+06,9.563601e+04,907.185035,7.863975e+04,9.097334e+04,1.152950e+06
min,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,2.000000e+03,3.000000e+01,2.000000,1.698000e+01,3.100000e+01,2.000000e+03
50%,5.200000e+03,6.200000e+02,12.000000,3.947200e+02,6.243300e+02,5.500000e+03
75%,1.600000e+04,4.076000e+03,56.000000,3.034090e+03,4.050000e+03,1.550000e+04
max,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08


In [175]:
data.describe(include='object')

,name,category,main_category,currency,deadline,launched,state,country
count,378657,378661,378661,378661,378661,378661,378661,378661
unique,375764,159,15,14,3164,378089,6,23
top,New EP/Music Development,Product Design,Film & Video,USD,2014-08-08,1970-01-01 01:00:00,failed,US
freq,41,22314,63585,295365,705,7,197719,292627


In [176]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378661 entries, 1000002330 to 999988282
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              378657 non-null  object 
 1   category          378661 non-null  object 
 2   main_category     378661 non-null  object 
 3   currency          378661 non-null  object 
 4   deadline          378661 non-null  object 
 5   goal              378661 non-null  float64
 6   launched          378661 non-null  object 
 7   pledged           378661 non-null  float64
 8   state             378661 non-null  object 
 9   backers           378661 non-null  int64  
 10  country           378661 non-null  object 
 11  usd pledged       374864 non-null  float64
 12  usd_pledged_real  378661 non-null  float64
 13  usd_goal_real     378661 non-null  float64
dtypes: float64(5), int64(1), object(8)
memory usage: 43.3+ MB


In [177]:
data.isna().sum()

name                   4
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         3797
usd_pledged_real       0
usd_goal_real          0
dtype: int64

We have some missing values, which we will impute in the next section

# Data preparation and cleaning

Let's make the deadline feature and launched features datetime fields

In [178]:
data['deadline'] = pd.to_datetime(data['deadline'])
data['launched'] = pd.to_datetime(data['launched'])

Here, I will get rid of missing values and drop features which are not explanatory 

In [179]:
data.columns

Index(['name', 'category', 'main_category', 'currency', 'deadline', 'goal',
       'launched', 'pledged', 'state', 'backers', 'country', 'usd pledged',
       'usd_pledged_real', 'usd_goal_real'],
      dtype='object')

Well, definitely, the feature name doesn't have any explanatory power

In [180]:
data.drop(['name'], inplace=True, axis=1)

Now let's impute missing values in usd_pledged

In [181]:
data['usd pledged'].fillna(data['usd pledged'].mean(), inplace=True)
data['usd pledged'].isna().sum()

0

Also let's look at state column, which occurs to be out target

In [182]:
data['state'].unique()

array(['failed', 'canceled', 'successful', 'live', 'undefined',
       'suspended'], dtype=object)

So it looks like successful and live are some state of successful obviously, and others are some case of failure

In [183]:
data['state'].replace('canceled', 'failed', inplace=True)
data['state'].replace('live', 'successful', inplace=True)
data['state'].replace('undefined', 'failed', inplace=True)
data['state'].replace('suspended', 'failed', inplace=True)
data['state'].unique()

array(['failed', 'successful'], dtype=object)

And let's split the dataset into y and X, which are correspondingly target and features

In [184]:
y = data.pop('state')
X = data.copy()

In [185]:
y.head()

ID
1000002330    failed
1000003930    failed
1000004038    failed
1000007540    failed
1000011046    failed
Name: state, dtype: object

In [186]:
X.head()

,category,main_category,currency,deadline,goal,launched,pledged,backers,country,usd pledged,usd_pledged_real,usd_goal_real
ID,,,,,,,,,,,,
1000002330,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,0,GB,0.0,0.0,1533.95
1000003930,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,15,US,100.0,2421.0,30000.00
1000004038,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,3,US,220.0,220.0,45000.00
1000007540,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,1,US,1.0,1.0,5000.00
1000011046,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,14,US,1283.0,1283.0,19500.00


# Feature engineering, encoding and scaling

Let's make the datetime columns comprehensible to the model

In [187]:
X['deadline_year'] = X['deadline'].dt.year.astype(np.float64)
X['deadline_month'] = X['deadline'].dt.month.astype(np.float64)

X['launched_year'] = X['launched'].dt.year.astype(np.float64)
X['launched_month'] = X['launched'].dt.month.astype(np.float64)

X['project_time_length'] = (X['deadline'] - X['launched']).dt.days.astype(np.float64)
X.drop(['deadline', 'launched'], inplace=True, axis=1)
X.head(10)

,category,main_category,currency,goal,pledged,backers,country,usd pledged,usd_pledged_real,usd_goal_real,deadline_year,deadline_month,launched_year,launched_month,project_time_length
ID,,,,,,,,,,,,,,,
1000002330,Poetry,Publishing,GBP,1000.0,0.00,0,GB,0.00,0.00,1533.95,2015.0,10.0,2015.0,8.0,58.0
1000003930,Narrative Film,Film & Video,USD,30000.0,2421.00,15,US,100.00,2421.00,30000.00,2017.0,11.0,2017.0,9.0,59.0
1000004038,Narrative Film,Film & Video,USD,45000.0,220.00,3,US,220.00,220.00,45000.00,2013.0,2.0,2013.0,1.0,44.0
1000007540,Music,Music,USD,5000.0,1.00,1,US,1.00,1.00,5000.00,2012.0,4.0,2012.0,3.0,29.0
1000011046,Film & Video,Film & Video,USD,19500.0,1283.00,14,US,1283.00,1283.00,19500.00,2015.0,8.0,2015.0,7.0,55.0
1000014025,Restaurants,Food,USD,50000.0,52375.00,224,US,52375.00,52375.00,50000.00,2016.0,4.0,2016.0,2.0,34.0
1000023410,Food,Food,USD,1000.0,1205.00,16,US,1205.00,1205.00,1000.00,2014.0,12.0,2014.0,12.0,19.0
1000030581,Drinks,Food,USD,25000.0,453.00,40,US,453.00,453.00,25000.00,2016.0,3.0,2016.0,2.0,44.0
1000034518,Product Design,Design,USD,125000.0,8233.00,58,US,8233.00,8233.00,125000.00,2014.0,5.0,2014.0,4.0,34.0


For encoding of the target I am going to use sklearn's ordinal encoder

In [188]:
from sklearn.preprocessing import OrdinalEncoder 
target_encoder = OrdinalEncoder()
y = pd.Series(target_encoder.fit_transform(y.values.reshape(-1,1)).reshape(1,-1)[0], index=y.index)
y.head(10)

ID
1000002330    0.0
1000003930    0.0
1000004038    0.0
1000007540    0.0
1000011046    0.0
1000014025    1.0
1000023410    1.0
1000030581    0.0
1000034518    0.0
100004195     0.0
dtype: float64

For others I am going to use one hot encoder

In [189]:
from sklearn.preprocessing import OneHotEncoder

cat_cols = [x for x in X.columns if X[x].dtype == 'object']
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoded = pd.DataFrame(encoder.fit_transform(X.loc[:, cat_cols]), index=X.index)
X.drop(cat_cols, inplace=True, axis=1)
X = pd.concat([X, encoded], axis=1)
X.head()


,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real,deadline_year,deadline_month,launched_year,launched_month,...,201,202,203,204,205,206,207,208,209,210
ID,,,,,,,,,,,,,,,,,,,,,
1000002330,1000.0,0.0,0,0.0,0.0,1533.95,2015.0,10.0,2015.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000003930,30000.0,2421.0,15,100.0,2421.0,30000.00,2017.0,11.0,2017.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1000004038,45000.0,220.0,3,220.0,220.0,45000.00,2013.0,2.0,2013.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1000007540,5000.0,1.0,1,1.0,1.0,5000.00,2012.0,4.0,2012.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1000011046,19500.0,1283.0,14,1283.0,1283.0,19500.00,2015.0,8.0,2015.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


For scaling purposes I am going to use Standard Scaler

In [190]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)
X.head()

,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real,deadline_year,deadline_month,launched_year,launched_month,...,201,202,203,204,205,206,207,208,209,210
ID,,,,,,,,,,,,,,,,,,,,,
1000002330,-0.040630,-0.101248,-0.116423,-0.089933,-0.099578,-0.038094,0.352985,0.981071,0.389063,0.461979,...,-0.010278,-0.012797,-0.068179,-0.100643,-0.087361,-0.043281,-0.061936,-0.068276,-0.038312,-1.844260
1000003930,-0.016124,-0.075934,-0.099889,-0.088655,-0.072966,-0.013404,1.392245,1.280244,1.423568,0.762268,...,-0.010278,-0.012797,-0.068179,-0.100643,-0.087361,-0.043281,-0.061936,-0.068276,-0.038312,0.542223
1000004038,-0.003448,-0.098948,-0.113117,-0.087121,-0.097160,-0.000394,-0.686274,-1.412315,-0.645443,-1.640042,...,-0.010278,-0.012797,-0.068179,-0.100643,-0.087361,-0.043281,-0.061936,-0.068276,-0.038312,0.542223
1000007540,-0.037250,-0.101238,-0.115321,-0.089920,-0.099567,-0.035088,-1.205904,-0.813969,-1.162696,-1.039464,...,-0.010278,-0.012797,-0.068179,-0.100643,-0.087361,-0.043281,-0.061936,-0.068276,-0.038312,0.542223
1000011046,-0.024997,-0.087833,-0.100991,-0.073535,-0.085475,-0.022511,0.352985,0.382724,0.389063,0.161690,...,-0.010278,-0.012797,-0.068179,-0.100643,-0.087361,-0.043281,-0.061936,-0.068276,-0.038312,0.542223


# Modeling

Let's split the data into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X,y, shuffle=True, random_state=1)

And now let's train the model!

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=[X.shape[1]]),
    layers.Dense(512, activation='relu'),  
    layers.Dense(256, activation='relu'),  
    layers.Dense(1, activation='sigmoid'),
])